<a href="https://colab.research.google.com/github/MrsIgnis/MOCI/blob/main/MOCI_task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**I. Создать нейронную сеть с нуля, т. е. не используя готовые библиотеки. Пример работы на любом табличном датасете**

In [2]:
import numpy as np

In [3]:
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))

In [4]:
def forward_pass(
    X: np.ndarray,
    W1: np.ndarray, b1: np.ndarray,
    W2: np.ndarray, b2: np.ndarray,
    W3: np.ndarray, b3: np.ndarray
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:

    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)

    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)

    Z3 = np.dot(A2, W3) + b3
    A3 = sigmoid(Z3)

    return A1, A2, A3

In [5]:
def initialize_weights(
    input_neurons: int, hidden_neurons_1: int,
    hidden_neurons_2: int, output_neurons: int
) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:

    np.random.seed(42)
    W1 = np.random.randn(input_neurons, hidden_neurons_1) * 0.1
    b1 = np.zeros((1, hidden_neurons_1))

    W2 = np.random.randn(hidden_neurons_1, hidden_neurons_2) * 0.1
    b2 = np.zeros((1, hidden_neurons_2))

    W3 = np.random.randn(hidden_neurons_2, output_neurons) * 0.1
    b3 = np.zeros((1, output_neurons))

    return W1, b1, W2, b2, W3, b3

In [6]:
def predict(
    X: np.ndarray,
    W1: np.ndarray, b1: np.ndarray,
    W2: np.ndarray, b2: np.ndarray,
    W3: np.ndarray, b3: np.ndarray
) -> np.ndarray:

    _, _, A3 = forward_pass(X, W1, b1, W2, b2, W3, b3)
    return A3

In [7]:
dataset = np.array([
    [22, 2, 1, 1, 38000],
    [27, 3, 2, 1, 42000],
    [34, 5, 4, 2, 47000],
    [40, 8, 5, 3, 53000],
    [45, 10, 6, 4, 60000],
    [50, 12, 6, 5, 62000],
    [26, 3, 3, 2, 44000],
    [36, 7, 4, 3, 49000],
    [42, 9, 5, 4, 55000],
    [48, 11, 6, 4, 58000]
])

In [8]:
X = dataset[:, :-1]
y = dataset[:, -1].reshape(-1, 1) / 100000

In [9]:
input_neurons = X.shape[1]
hidden_neurons_1 = 10
hidden_neurons_2 = 6
output_neurons = 1

In [10]:
W1, b1, W2, b2, W3, b3 = initialize_weights(input_neurons, hidden_neurons_1, hidden_neurons_2, output_neurons)

In [11]:
predictions = predict(X, W1, b1, W2, b2, W3, b3)
predictions = predictions * 100000

In [12]:
print(f"{'Возраст':<10} {'Опыт':<10} {'Образование':<15} {'Стаж':<10} {'Доход в датасете':<20} {'Предсказанный доход':<20}")
for i in range(len(X)):
    print(f"{X[i][0]:<10} {X[i][1]:<10} {X[i][2]:<15} {X[i][3]:<10} {y[i][0]*100000:<20} {predictions[i][0]:<20.2f}")

Возраст    Опыт       Образование     Стаж       Доход в датасете     Предсказанный доход 
22         2          1               1          38000.0              46491.85            
27         3          2               1          42000.0              46498.57            
34         5          4               2          47000.0              46513.98            
40         8          5               3          53000.0              46526.87            
45         10         6               4          60000.0              46533.71            
50         12         6               5          62000.0              46539.25            
26         3          3               2          44000.0              46504.06            
36         7          4               3          49000.0              46525.35            
42         9          5               4          55000.00000000001    46533.64            
48         11         6               4          57999.99999999999    46534.30            

**II. Сделать класс, в котором реализована возможность задать количество нейронов какого-то из слоёв, и провести обучение**

In [223]:
import pandas as pd

In [224]:
df = pd.read_csv('/content/pokemon_dataset.csv')

In [225]:
df

,Pokemon,Level,Attack,Defense,Speed,Health
0,Bulbasaur,5,49,49,45,318
1,Ivysaur,16,62,63,60,405
2,Venusaur,32,82,83,80,525
3,Charmander,5,52,43,65,309
4,Charmeleon,16,64,58,80,405
5,Charizard,36,84,78,100,534
6,Squirtle,5,48,65,43,314
7,Wartortle,16,63,80,58,405
8,Blastoise,36,83,100,78,530
9,Pikachu,5,55,40,90,320


In [238]:
X = df.drop(['Pokemon', 'Health'], axis=1).values
y = df['Health'].values.reshape(-1, 1) / 1000
pokemon_names = df['Pokemon'].values

In [239]:
class NeuralNetwork:
    def __init__(self, input_size: int, hidden1_size: int, hidden2_size: int, output_size: int, learning_rate: float = 0.1) -> None:
        np.random.seed(42)
        self.lr = learning_rate

        self.W1 = np.random.randn(input_size, hidden1_size) * 0.1
        self.b1 = np.zeros((1, hidden1_size))

        self.W2 = np.random.randn(hidden1_size, hidden2_size) * 0.1
        self.b2 = np.zeros((1, hidden2_size))

        self.W3 = np.random.randn(hidden2_size, output_size) * 0.1
        self.b3 = np.zeros((1, output_size))

In [240]:
def sigmoid_derivative(x: np.ndarray) -> np.ndarray:
    return x * (1 - x)

In [241]:
def forward(nn: NeuralNetwork, X: np.ndarray) -> np.ndarray:
    nn.z1 = np.dot(X, nn.W1) + nn.b1
    nn.a1 = sigmoid(nn.z1)

    nn.z2 = np.dot(nn.a1, nn.W2) + nn.b2
    nn.a2 = sigmoid(nn.z2)

    nn.z3 = np.dot(nn.a2, nn.W3) + nn.b3
    nn.a3 = sigmoid(nn.z3)

    return nn.a3

In [242]:
def update_weights(nn: NeuralNetwork, X: np.ndarray, d_output: np.ndarray, d_z2: np.ndarray, d_z1: np.ndarray) -> None:
    nn.W3 += nn.a2.T.dot(d_output) * nn.lr
    nn.b3 += np.sum(d_output, axis=0, keepdims=True) * nn.lr

    nn.W2 += nn.a1.T.dot(d_z2) * nn.lr
    nn.b2 += np.sum(d_z2, axis=0, keepdims=True) * nn.lr

    nn.W1 += X.T.dot(d_z1) * nn.lr
    nn.b1 += np.sum(d_z1, axis=0, keepdims=True) * nn.lr

In [243]:
def backward(nn: NeuralNetwork, X: np.ndarray, y: np.ndarray, output: np.ndarray) -> None:
    error = y - output
    d_output = error * sigmoid_derivative(output)
    d_z2 = d_output.dot(nn.W3.T) * sigmoid_derivative(nn.a2)
    d_z1 = d_z2.dot(nn.W2.T) * sigmoid_derivative(nn.a1)

    update_weights(nn, X, d_output, d_z2, d_z1)

In [244]:
def train(nn: NeuralNetwork, X: np.ndarray, y: np.ndarray, epochs: int = 1000) -> None:
    for epoch in range(epochs):
        output = forward(nn, X)
        backward(nn, X, y, output)

In [245]:
def predict(nn: NeuralNetwork, X: np.ndarray) -> np.ndarray:
    return forward(nn, X)

In [246]:
nn = NeuralNetwork(
    input_size=4,  # количество признаков (Level, Attack, Defense, Speed)
    hidden1_size=10,
    hidden2_size=6,
    output_size=1,  # Прогнозируем только одно значение (Health)
    learning_rate=0.5
)

In [247]:
train(nn, X, y, epochs=1000)
predictions = predict(nn, X)

In [248]:
print(f"{'Pokemon':<15} {'Level':<10} {'Attack':<10} {'Defense':<15} {'Speed':<10} {'Health in dataset':<20} {'Predicted Health':<20}")
for i in range(len(X)):
    print(f"{pokemon_names[i]:<15} {X[i][0]:<10} {X[i][1]:<10} {X[i][2]:<15} {X[i][3]:<10} {y[i][0]*1000:<20.0f} {predictions[i][0]*1000:<20.2f}")

Pokemon         Level      Attack     Defense         Speed      Health in dataset    Predicted Health    
Bulbasaur       5          49         49              45         318                  415.00              
Ivysaur         16         62         63              60         405                  415.00              
Venusaur        32         82         83              80         525                  415.00              
Charmander      5          52         43              65         309                  415.00              
Charmeleon      16         64         58              80         405                  415.00              
Charizard       36         84         78              100        534                  415.00              
Squirtle        5          48         65              43         314                  415.00              
Wartortle       16         63         80              58         405                  415.00              
Blastoise       36         83        